In [1]:
import pandas as pd
import sys
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold
import json
from joblib import dump
from sklearn.model_selection import cross_val_score
from mrmr import mrmr_classif
from collections import Counter
import  matplotlib.pyplot as plt
import os
sys.path.append(os.path.abspath('..'))
from utils.get_splitted_data import get_tr_vl_ts_list

In [2]:
data = pd.read_csv('./Data/2D_t2/all.csv')
data["Label"] = data['Label'].apply(lambda x: x-1)
data["Name"] = data['Name'].str.replace(r'^IU_', 'IUC_', regex=True)
# data["Name"] = data['Name'].str.lower()

Apply T4: train = [AHN EMC IU MCA MCF NU], test=[NYU]

In [3]:
def get_results(acc_list,auc_list,recall_list,precision_list):    
    acc_list = np.array(acc_list)
    auc_list = np.array(auc_list)
    recall_list = np.array(recall_list)
    precision_list = np.array(precision_list)
    print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
    print(f'precision, Average:{precision_list.mean():.4f}, Std:{precision_list.std():.4f}')
    print(f'Accuracy, Average:{acc_list.mean():.4f}, Std:{acc_list.std():.4f}')
    print(f'AUC, Average:{auc_list.mean():.4f}, Std:{auc_list.std():.4f}')
    return auc_list.mean()

In [4]:
train_test_info = 'Train_Test_4'


In [5]:
dataset_dtl_path = '/Users/zilianghong/Documents/GitHub/IPMN-Radiomics-Plus-Deeplearning/Train_Test_4.json'
train_list,val_list,test_list = get_tr_vl_ts_list(dataset_dtl=dataset_dtl_path,fold=0)

In [6]:
train_data = data[data['Name'].isin(train_list)]
test_data = data[data['Name'].isin(test_list)]
val_data = data[data['Name'].isin(val_list)]
# For grid search
train_val_data = data[data['Name'].isin(val_list+train_list)]

In [7]:
# feature_list.sort()
feature_list = [
'skewness-Laws R5S5',
'Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_5',
'Collage_skew_InformationMeasureOfCorrelation2_1_nb_8_ws_3',
'Collage_kurt_Contrast_1_nb_8_ws_7',
'Collage_median_MaximalCorrelationCoefficient_1_nb_32_ws_3',
'Collage_skew_Correlation_1_nb_16_ws_5',
'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_3',
'Collage_skew_SumVariance_1_nb_8_ws_5',
'median-Laws E5L5',
'Collage_skew_Entropy_1_nb_16_ws_7',
'skewness-Laws S5E5',
'skewness-Laws W5L5',
'Collage_skew_DifferenceEntropy_1_nb_4_ws_3',
'Collage_var_Contrast_1_nb_8_ws_7',
'Collage_kurt_SumEntropy_1_nb_16_ws_5',
]

In [11]:
X = train_val_data.drop(columns=['Center','Name','Label'])
y = train_val_data[["Label"]]
standard_scaler = StandardScaler()
X_scaled = pd.DataFrame(standard_scaler.fit_transform(X),columns=X.columns)
X_selected = X_scaled[feature_list]

param_grid = {
    'n_estimators': [47,48,49],          
    'max_depth': [10,11,12,13,14,15],            
    'min_samples_split': [5],         
    'min_samples_leaf': [1],           
    'max_features': [None],  
    'bootstrap': [True]               
} 

# GridSearchCV get bset hyper parameters
rf_model = RandomForestClassifier(random_state=3)
grid_search = GridSearchCV(
    estimator=rf_model,           # 目标模型
    param_grid=param_grid,        # 参数网格
    cv=5,                         # 5折交叉验证
    scoring='roc_auc',           # 评分标准（可以选择'accuracy', 'f1'等）
    n_jobs=-1,                    # 使用所有可用内核
    verbose=1                     # 输出搜索过程
)


grid_search.fit(X_selected, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=3), n_jobs=-1,
             param_grid={'bootstrap': [True],
                         'max_depth': [10, 11, 12, 13, 14, 15],
                         'max_features': [None], 'min_samples_leaf': [1],
                         'min_samples_split': [5],
                         'n_estimators': [47, 48, 49]},
             scoring='roc_auc', verbose=1)

In [12]:
results = grid_search.cv_results_
for i in range(len(results['params'])):
    print(f"Search {i + 1}: {results['params'][i]}")
    mean_test_auc = results['mean_test_score'][i]
    print(f"Average AUC: {mean_test_auc:.4f}")
    
    for fold in range(grid_search.cv):
        # fold_test_accuracy = results[f'split{fold}_test_score'][i]
        fold_test_auc = results[f'split{fold}_test_score'][i]
        print(f"  Fold {fold + 1} - , validation AUC: {fold_test_auc:.4f}")
    print("-" * 40)

Search 1: {'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 47}
Average AUC: 0.7794
  Fold 1 - , validation AUC: 0.7953
  Fold 2 - , validation AUC: 0.8493
  Fold 3 - , validation AUC: 0.7203
  Fold 4 - , validation AUC: 0.7664
  Fold 5 - , validation AUC: 0.7658
----------------------------------------
Search 2: {'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 48}
Average AUC: 0.7835
  Fold 1 - , validation AUC: 0.7929
  Fold 2 - , validation AUC: 0.8480
  Fold 3 - , validation AUC: 0.7247
  Fold 4 - , validation AUC: 0.7803
  Fold 5 - , validation AUC: 0.7715
----------------------------------------
Search 3: {'bootstrap': True, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 49}
Average AUC: 0.7810
  Fold 1 - , validation AUC: 0.7917
  Fold 2 - , validation AUC: 0.8474
  Fold 3 - , valid

Test Set

In [13]:
x_test_raw = test_data.drop(columns=['Center','Name','Label'])
y_test = test_data[["Label"]]
x_test_scaled = pd.DataFrame(standard_scaler.transform(x_test_raw),columns=x_test_raw.columns)
x_test = x_test_scaled[feature_list]

In [15]:
models = []
test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=5)
for fold in range(5):
    train_list,val_list,test_list = get_tr_vl_ts_list(dataset_dtl=dataset_dtl_path,fold=fold)
    train_data = data[data['Name'].isin(train_list)]
    val_data = data[data['Name'].isin(val_list)]

    x_train_raw = train_data.drop(columns=['Center','Name','Label'])
    y_train = train_data[["Label"]]
    x_train_scaled = pd.DataFrame(standard_scaler.transform(x_train_raw),columns=x_train_raw.columns)
    x_train = x_train_scaled[feature_list]

    x_val_raw = val_data.drop(columns=['Center','Name','Label'])
    y_val = val_data[["Label"]]
    x_val_scaled = pd.DataFrame(standard_scaler.transform(x_val_raw),columns=x_val_raw.columns)
    x_val = x_val_scaled[feature_list]
    
    # Get best hyper params
    best_params = grid_search.best_params_
    best_rf_model = RandomForestClassifier(**best_params, random_state=42)
    best_rf_model.fit(x_train, y_train)
    # Save model
    models.append(best_rf_model)
    
    fold_accuracy = accuracy_score(y_val, best_rf_model.predict(x_val))
    val_acc_list.append(fold_accuracy)
    fold_auc = roc_auc_score(y_val, best_rf_model.predict_proba(x_val)[:, 1])
    val_auc_list.append(fold_auc)
    print(f"Fold {fold + 1} - ACC: {fold_accuracy:.4f}, AUC: {fold_auc:.4f}")
    test_accuracy = accuracy_score(y_test, best_rf_model.predict(x_test))
    test_acc_list.append(test_accuracy)
    test_auc = roc_auc_score(y_test, best_rf_model.predict_proba(x_test)[:, 1])
    test_auc_list.append(test_auc)
    print(f"Test {fold + 1} - ACC: {test_accuracy:.4f}, AUC: {test_auc:.4f}")
    print("-" * 40)
print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
print("Test set")
get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')


Fold 1 - ACC: 0.7414, AUC: 0.7333
Test 1 - ACC: 0.7042, AUC: 0.7968
----------------------------------------
Fold 2 - ACC: 0.6897, AUC: 0.8144
Test 2 - ACC: 0.6901, AUC: 0.7987
----------------------------------------
Fold 3 - ACC: 0.6724, AUC: 0.6919
Test 3 - ACC: 0.6901, AUC: 0.7681
----------------------------------------
Fold 4 - ACC: 0.6034, AUC: 0.7525
Test 4 - ACC: 0.6901, AUC: 0.7982
----------------------------------------
Fold 5 - ACC: 0.7091, AUC: 0.7323
Test 5 - ACC: 0.7042, AUC: 0.7746
----------------------------------------
Validation set
Recall, Average:nan, Std:nan
precision, Average:nan, Std:nan
Accuracy, Average:0.6832, Std:0.0460
AUC, Average:0.7449, Std:0.0400
Test set
Recall, Average:nan, Std:nan
precision, Average:nan, Std:nan
Accuracy, Average:0.6958, Std:0.0069
AUC, Average:0.7873, Std:0.0132


Save model

In [16]:
root = './trained_models'
save_path = os.path.join(root,train_test_info)
if not os.path.exists(save_path):
    os.mkdir(save_path)
for index,model in enumerate(models):
    dump(model, os.path.join(save_path,f'{train_test_info}_Fold{index+1}.joblib'))